In [ ]:
%pip install -r requirements.txt

In [8]:
# import installed libraries
from dotenv import load_dotenv
import os
import json
from openai import AzureOpenAI
import re


# load environment variables
load_dotenv(override=True)


True

In [21]:
# loading the markdown file
markdown_file = os.path.join(os.getcwd(), "data", "DRAFT_Acolad_2023 - English.md")
print(markdown_file)
with open(markdown_file, 'r') as file:
    data = file.read()

c:\Users\szetinglau\Documents\Github\IFU-translation\data\DRAFT_Acolad_2023 - English.md


In [25]:
import re
import textwrap

def chunk_markdown(text, chunk_size=1000):
    """Chunks Markdown text while preserving structure (paragraphs & formatting)."""
    paragraphs = re.split(r'(\n\s*\n)', text)  # Split by double newlines (paragraphs)

    chunks = []
    current_chunk = ""

    for para in paragraphs:
        if len(current_chunk) + len(para) > chunk_size:
            chunks.append(current_chunk.strip())
            current_chunk = para
        else:
            current_chunk += para

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

data = chunk_markdown(data, 1000)
print(data)

['<figure>\n\n<!-- PageHeader="T Instructions for Use" -->\n\n</figure>\n\n\nen\n\nSTRATAFIXâ„¢ Spiral MONOCRYLâ„¢ Plus\nKnotless Tissue Control Device\nAntibacterial\nUndyed Monofilament Synthetic\nAbsorbable Unidirectional Device\n\n\n# Description\n\nSTRATAFIXâ„¢ Spiral MONOCRYLâ„¢ Plus Knotless Tissue Control Device is a sterile,\nantibacterial monofilament, synthetic absorbable device composed of 100\nweight percent copolymer of glycolide and E-caprolactone. The empirical\nformula of the copolymer is (C, H, O,)_(C H O) . Poliglecaprone 25 copolymer\nhas been found to be nonpyrogenic and elicits only a slight tissue reaction\nduring absorption.', 'STRATAFIXâ„¢ Spiral MONOCRYLâ„¢ Plus Device contains IRGACAREÂ®+ MP\n(triclosan), a broad spectrum antibacterial agent, at no more than 2360 Âµg/m.\nSTRATAFIXâ„¢ Spiral MONOCRYLâ„¢ Plus Device, Variable Loop Design consists\nof barbed suture material, armed with a surgical needle on one end and a\nfixation loop at the opposite end. STRATA

### Initializing the Azure OpenAI Client

In [26]:
# define openai Client
aoai_api_key = os.getenv("AZURE_OPENAI_KEY")
aoai_api_endpoint =  os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

client = AzureOpenAI(
    api_key=aoai_api_key,
    api_version="2024-08-01-preview",
    azure_endpoint=aoai_api_endpoint
)

In [34]:
translated_chunks = []
for chunk in data:
    systemPrompt = f"""
    Your Task is to translate the given text from English to Spanish:
    Task:
        Step 1: Translate the given text to Spanish, ensuring that the meaning (semantics) of the original text is preserved as closely as possible.
        Step 2: Do not translate any text containing "trademark symbols" (™ or ®). Keep these parts as is.
        Below is the text for translation:
        ----------------------------------------

        {chunk}
    """

    conversaion = [
        {"role": "system", "content": systemPrompt},
        {"role": "user", "content": "give me the translation in Spanish"}
    ]

    # Send the conversation to the API
    response = client.chat.completions.create(
        model=aoai_api_deployment_name, # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
        messages=conversaion,
        temperature=0.0,
    )

    translated_chunks.append(response.choices[0].message.content)

final_translation = "\n\n".join(translated_chunks) 
print(final_translation)

STRATAFIX™ Spiral MONOCRYL™ Plus
Dispositivo de Control de Tejidos sin Nudos
Antibacteriano
Monofilamento Sintético No Teñido
Dispositivo Absorbible Unidireccional

# Descripción

El dispositivo de control de tejidos sin nudos STRATAFIX™ Spiral MONOCRYL™ Plus es un dispositivo absorbible sintético monofilamento antibacteriano estéril, compuesto por un 100 por ciento en peso de copolímero de glicólido y E-caprolactona. La fórmula empírica del copolímero es (C, H, O,)_(C H O). Se ha encontrado que el copolímero de poliglecaprone 25 es no pirogénico y provoca solo una leve reacción tisular durante la absorción.

El dispositivo STRATAFIX™ Spiral MONOCRYL™ Plus contiene IRGACARE® MP (triclosan), un agente antibacteriano de amplio espectro, con no más de 2360 µg/m. El dispositivo STRATAFIX™ Spiral MONOCRYL™ Plus, con diseño de bucle variable, consiste en material de sutura con barbas, equipado con una aguja quirúrgica en un extremo y un bucle de fijación en el extremo opuesto. El dispositivo

In [35]:
# save the responseText to a file with utf-8 encoding
output_file = os.path.join(os.getcwd(), "data", "DRAFT_Acolad_2023 - Spanish.md")
with open(output_file, 'w', encoding='utf-8') as file:
    file.write(final_translation)

In [36]:
# loading the markdown file
markdown_file = os.path.join(os.getcwd(), "data", "DRAFT_Acolad_2023 - Spanish.md")
print(markdown_file)
with open(markdown_file, 'r') as file:
    data = file.read()

c:\Users\szetinglau\Documents\Github\IFU-translation\data\DRAFT_Acolad_2023 - Spanish.md


In [37]:
data = chunk_markdown(data, 1000)

In [38]:
translated_chunks = []
for chunk in data:
    systemPrompt = f"""
    Your Task is to translate the given text from Spanish to English:
    Task:
        Step 1: Translate the given text to English, ensuring that the meaning (semantics) of the original text is preserved as closely as possible.
        Step 2: Do not translate any text containing "trademark symbols" (™ or ®). Keep these parts as is.
        Below is the text for translation:
        ----------------------------------------

        {chunk}
    """

    conversaion = [
        {"role": "system", "content": systemPrompt},
        {"role": "user", "content": "give me the translation in English"}
    ]

    # Send the conversation to the API
    response = client.chat.completions.create(
        model=aoai_api_deployment_name, # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
        messages=conversaion,
        temperature=0.0,
    )

    translated_chunks.append(response.choices[0].message.content)

final_translation = "\n\n".join(translated_chunks) 
print(final_translation)

STRATAFIX™ Spiral MONOCRYL™ Plus
Knotless Tissue Control Device
Antibacterial
Undyed Synthetic Monofilament
Unidirectional Absorbable Device

# Description

The STRATAFIX™ Spiral MONOCRYL™ Plus knotless tissue control device is a sterile antibacterial synthetic absorbable monofilament device, composed of 100 percent by weight of glycolide and E-caprolactone copolymer. The empirical formula of the copolymer is (C, H, O,)_(C H O). The poliglecaprone 25 copolymer has been found to be non-pyrogenic and causes only a mild tissue reaction during absorption.

The STRATAFIX™ Spiral MONOCRYL™ Plus device contains IRGACARE® MP (triclosan), a broad-spectrum antibacterial agent, with no more than 2360 µg/m. The STRATAFIX™ Spiral MONOCRYL™ Plus device, with a variable loop design, consists of barbed suture material equipped with a surgical needle at one end and a fixation loop at the opposite end. The STRATAFIX™ Spiral MONOCRYL™ Plus device is designed to anchor with a closed loop at one end and a 

In [39]:
# save the responseText to a file with utf-8 encoding
output_file = os.path.join(os.getcwd(), "data", "DRAFT_Acolad_2023_V2 - English.md")
with open(output_file, 'w', encoding='utf-8') as file:
    file.write(final_translation)